In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
rna_tsne = '../../../tybalt/results/rnaseq_tsne_features.tsv'
rna_nmf = '../../nmf_rnaseq.tsv'
rna_pca = '../../pca_rnaseq.tsv'
rna_ica = '../../ica_rnaseq.tsv'
rna_vae = '../../../tybalt/data/encoded_rnaseq_onehidden_warmup_batchnorm.tsv'
rna_adage = '../../../tybalt/data/encoded_adage_features.tsv'

# copy_pca = 'CopyNumber_pca_results.tsv'
# copy_tsne = 'CopyNumber_tsne_results.tsv'
# copy_nmf = 'CopyNumber_nmf_results.tsv'

clinical = "vae_encoded_with_clinical.tsv"

In [3]:
rna_tsne_df = pd.read_table(rna_tsne, index_col=0)
rna_nmf_df = pd.read_table(rna_nmf, index_col=0)
rna_pca_df = pd.read_table(rna_pca, index_col=0)
rna_ica_df = pd.read_table(rna_ica, index_col=0)
rna_vae_df = pd.read_table(rna_vae, index_col=0)
rna_adage_df = pd.read_table(rna_adage, index_col=0)

# copy_pca_df = pd.read_table(copy_pca, index_col=0)
# copy_tsne_df = pd.read_table(copy_tsne, index_col=0)
# copy_nmf_df = pd.read_table(copy_nmf, index_col=0)

clinical_df = pd.read_table(clinical, index_col=0)

In [4]:
real_clinical_df = clinical_df.iloc[:, range(101, 117)]
real_clinical_df.index = clinical_df.index

# Fix some clinical variables
real_clinical_df["year_of_diagnosis"] = real_clinical_df["year_of_diagnosis"].fillna(0)
real_clinical_df["year_of_diagnosis"] = real_clinical_df["year_of_diagnosis"].astype(int)
real_clinical_df.loc[(real_clinical_df["year_of_diagnosis"] < 1979), ["year_of_diagnosis"]] = np.nan

# Assign diagnosis year and age to bins
year_interval = pd.IntervalIndex.from_tuples([
        (1978, 2003),
        (2003, 2007),
        (2007, 2009),
        (2009, 2010),
        (2010, 2011),
        (2011, 2013)
    ])

age_interval = pd.IntervalIndex.from_tuples([
        (0, 34),
        (34, 44),
        (44, 50),
        (50, 55),
        (55, 60),
        (60, 65),
        (65, 70),
        (70, 75),
        (75, 95)
    ])

real_clinical_df["age_at_diagnosis"] = pd.cut(real_clinical_df["age_at_diagnosis"], bins = age_interval)
real_clinical_df["year_of_diagnosis"] = pd.cut(real_clinical_df["year_of_diagnosis"], bins = year_interval)

/home/gway/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gway/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gway/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

In [5]:
rna_tsne_df = rna_tsne_df.add_suffix('_rna_tsne')
rna_nmf_df = rna_nmf_df.add_suffix('_rna_nmf')
rna_pca_df = rna_pca_df.add_suffix('_rna_pca')
rna_ica_df = rna_ica_df.add_suffix('_rna_ica')
rna_vae_df = rna_vae_df.add_suffix('_rna_vae')
rna_adage_df = rna_adage_df.add_suffix('_rna_adage')

# copy_pca_df = copy_pca_df.add_suffix('_copy_pca')
# copy_tsne_df = copy_tsne_df.add_suffix('_copy_tsne')
# copy_nmf_df = copy_nmf_df.add_suffix('_copy_nmf')

In [6]:
all_data_combined = (rna_tsne_df.merge(rna_nmf_df, how='inner', left_index=True, right_index=True)
                     .merge(rna_pca_df, how='inner', left_index=True, right_index=True)
                     .merge(rna_ica_df, how='inner', left_index=True, right_index=True)
                     .merge(rna_vae_df, how='inner', left_index=True, right_index=True)
                     .merge(rna_adage_df, how='inner', left_index=True, right_index=True)
                     # .merge(copy_pca_df, how='inner', left_index=True, right_index=True)
                     # .merge(copy_tsne_df, how='inner', left_index=True, right_index=True)
                     # .merge(copy_nmf_df, how='inner', left_index=True, right_index=True)
                     .merge(real_clinical_df, how='inner', left_index=True, right_index=True))

In [7]:
all_data_combined.to_csv('data_feature_encodings.tsv', sep='\t')